In [31]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import math 
from matplotlib.figure import Figure
pd.set_option('display.max_colwidth', None)

In [56]:
file=pd.read_excel('Bank_Personal_Loan_Modelling.xlsx', sheet_name='Data')

In [57]:
details=pd.read_excel('Bank_Personal_Loan_Modelling.xlsx')

In [58]:
file = file.drop('Experience', axis=1)
index = file[["ID"]]
file = file.drop('ID', axis=1)

In [59]:
y=file['Personal Loan']

In [60]:
X=file.drop('Personal Loan', axis='columns')

In [61]:
X

,Age,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,25,49,91107,4,1.6,1,0,1,0,0,0
1,45,34,90089,3,1.5,1,0,1,0,0,0
2,39,11,94720,1,1.0,1,0,0,0,0,0
3,35,100,94112,1,2.7,2,0,0,0,0,0
4,35,45,91330,4,1.0,2,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
4995,29,40,92697,1,1.9,3,0,0,0,1,0
4996,30,15,92037,4,0.4,1,85,0,0,1,0
4997,63,24,93023,2,0.3,3,0,0,0,0,0
4998,65,49,90034,3,0.5,2,0,0,0,1,0


In [62]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

!pip install category_encoders
%matplotlib inline

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [64]:

print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (4000, 11) (4000,)
Test (1000, 11) (1000,)


In [65]:
print(X_train)

      Age  Income  ZIP Code  Family  CCAvg  Education  Mortgage  \
868    40     161     94596       2    3.3          1         0   
936    62      19     92109       1    1.5          3         0   
4034   35      82     94022       2    1.7          2         0   
1923   45      22     90639       1    0.2          1         0   
3330   34      32     95054       4    1.1          3         0   
...   ...     ...       ...     ...    ...        ...       ...   
3335   35     118     92069       2    7.8          1       358   
1099   30      52     92717       3    0.7          2         0   
2514   41      25     92182       2    0.1          2        91   
3606   43       9     96145       2    0.0          3        96   
2575   42      41     90401       3    0.5          3         0   

      Securities Account  CD Account  Online  CreditCard  
868                    1           0       0           1  
936                    1           0       0           0  
4034              


## Hyperparameter tuning

In [66]:
#example of grid searching key hyperparametres for logistic regression
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
#define dataset
#X, y = make_blobs(n_samples=1000, centers=2, n_features=100, cluster_std=20)
#define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
#define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='recall',error_score=0)
grid_result = grid_search.fit(X_train_enc, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.635965 using {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.635965 (0.076360) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.352317 (0.063027) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.348920 (0.062717) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.635110 (0.074703) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.367117 (0.093776) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.351484 (0.061847) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.625686 (0.068303) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.372830 (0.085367) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.349775 (0.063022) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.593162 (0.072867) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.369433 (0.067764) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.348920 (0.062717) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.512506 (0.071844) wit

/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "


In [67]:
logmodel=LogisticRegression(C=1.0, penalty='l2',solver='newton-cg')

In [68]:
logmodel.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "


LogisticRegression(solver='newton-cg')

In [69]:

predictions=logmodel.predict(X_test)

In [70]:
#predictions

In [71]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [72]:
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n           0       0.96      0.99      0.97       909\n           1       0.80      0.56      0.66        91\n\n    accuracy                           0.95      1000\n   macro avg       0.88      0.77      0.81      1000\nweighted avg       0.94      0.95      0.94      1000\n'

In [73]:
confusion_matrix(y_test,predictions)

array([[896,  13],
       [ 40,  51]])

In [74]:
accuracy_score(y_test,predictions)

0.947

In [75]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [76]:
print('The Precision Score- What proportion of positive identifications was actually correct?: ' + str(100*precision_score(y_test,predictions))+'%')
print('The Recall Score - What proportion of actual positives was identified correctly?: ' + str(100*recall_score(y_test,predictions))+'%')

The Precision Score- What proportion of positive identifications was actually correct?: 79.6875%
The Recall Score - What proportion of actual positives was identified correctly?: 56.043956043956044%


In [77]:
scores = cross_val_score(logmodel, X_train, y_train, cv=15)
#print('Cross-Validation Accuracy Scores', scores.mean())
recall = cross_val_score(logmodel, X_train, y_train, cv=15, scoring='recall')
#print('Recall', recall.mean())
precision = cross_val_score(logmodel, X_train, y_train, cv=15, scoring='precision')
#print('Precision', precision.mean())

/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn(

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn(

/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iteration

/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iteration

/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn(

/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn(

In [78]:
scores.mean()

0.9515042287366355

In [79]:
recall.mean()

0.6427692307692306

In [80]:
precision.mean()

0.8206413809265473

# Accuracy: 95.15%
# Precision 82.06%
# Recall: 64.27%

# 
# 
# 
# 
# 
# 
# 
